# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Always state the H0 and H1 before conducting the hypothesis test.
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np

# importing all required modules here (beginning of file)
import scipy.stats
from scipy.stats import ttest_1samp
from statsmodels.stats.proportion import proportions_ztest

from helper_functions import get_confidence_interval

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [2]:
# Your code here:
salaries = pd.read_csv('Current_Employee_Names_Salaries_and_Position_Titles.csv')
salaries.columns = salaries.columns.str.lower().str.replace(' ', '_')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Y'our code here:
salaries.head()

,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
salaries.isna().sum()


name                     0
job_titles               0
department               0
full_or_part-time        0
salary_or_hourly         0
typical_hours        25528
annual_salary         8174
hourly_rate          25528
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
salaries['salary_or_hourly'].value_counts()

salary_or_hourly
Salary    25528
Hourly     8174
Name: count, dtype: int64

> no annual_salary when hourly pay   
> no typicial_hours and hourls_rate when salaried  

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
salaries.groupby('department').size().reset_index()
#https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.size.html

,department,0
0,ADMIN HEARNG,38
1,ANIMAL CONTRL,75
2,AVIATION,1856
3,BOARD OF ELECTION,111
4,BOARD OF ETHICS,8
5,BUDGET & MGMT,44
6,BUILDINGS,258
7,BUSINESS AFFAIRS,167
8,CITY CLERK,88
9,CITY COUNCIL,362


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# Your code here:
# import was done at the beginning of the file 
# get dataset -- hourly_rate from all hourly workers
hourly = salaries[salaries['salary_or_hourly'] == 'Hourly']['hourly_rate']
get_confidence_interval(hourly, 0.95)

(34.29839539204317, 34.874504045196865)

> we select a value for alpha of 0.05 (p-value threshold, significance level)    
> H0: mean hourly_rate = 30  
> H1: mean hourly_rate <> 30

In [8]:
stat, pval = ttest_1samp(hourly, 30)
print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)

stat is   31.211500329216044
pvalue for the two-tailed test is  3.825475847673794e-202


> p-value is very small, indicating evidence against the null hypothesis   
> **I would reject H0**

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our Null hypothesis is less than or equal to, then to reject, the test statistic must be positive (rejects the <) and the p-value below 0.05 (rejects the =).

In [9]:
# Your code here:
# get dataset -- annual_salary from all salaried workers
salaried_police = salaries[(salaries['salary_or_hourly'] == 'Salary') & (salaries['department'] == 'POLICE')]['annual_salary']
get_confidence_interval(salaried_police, 0.95)


(88534.1182885883, 89134.11955863392)

> we select a value for alpha of 0.05 (p-value threshold, significance level)    
> H0: mean annual_salary <= 86000  
> H1: mean annual_salary > 86000

In [10]:
stat, pval = ttest_1samp(salaried_police, 86000)
print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval/2)

stat is   18.51748561981861
pvalue for the two-tailed test is  6.119566952121482e-76


> p-value is very small, indicating evidence against the null hypothesis    
> further: stat is pointing in correct direction   
> **I would reject H0**

Using the `crosstab` function, find the department that has the most hourly workers. 

In [11]:
# Your code here:
cross_tab = pd.crosstab(salaries['department'], salaries['salary_or_hourly'])
display(cross_tab)
department_highest_hourly = cross_tab['Hourly'].idxmax()
print('Most hourly worker in department: ',department_highest_hourly)

salary_or_hourly,Hourly,Salary
department,,
ADMIN HEARNG,0,38
ANIMAL CONTRL,12,63
AVIATION,1214,642
BOARD OF ELECTION,0,111
BOARD OF ETHICS,0,8
BUDGET & MGMT,2,42
BUILDINGS,0,258
BUSINESS AFFAIRS,7,160
CITY CLERK,0,88


Most hourly worker in department:  STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

> we select a value for alpha of 0.05 (p-value threshold, significance level)    
> H0: mean hourly_wage >= 35  
> H1: mean hourly_wage < 35

In [12]:
# Your code here:
# get dataset -- hourly_rate from all salaried workers
hourly_department = salaries[(salaries['salary_or_hourly'] == 'Hourly') & (salaries['department'] == department_highest_hourly)]['hourly_rate']
display(get_confidence_interval(hourly_department, 0.95))

stat, pval = ttest_1samp(hourly_department, 86000)
print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval/2)

(35.28150811892106, 35.82308559722194)

stat is   -622617.5615407362
pvalue for the two-tailed test is  0.0


> p-value is zero small, indicating evidence against the null hypothesis   
> further: stat is pointing in correct direction   
> **I would reject H0**

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [13]:
# Your code here:
# see helper_function.py

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [14]:
# Your code here:
salaried_police = salaries[(salaries['salary_or_hourly'] == 'Salary') & (salaries['department'] == 'POLICE')]['annual_salary']
get_confidence_interval(salaried_police, 0.95)

(88534.1182885883, 89134.11955863392)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [15]:
# Your code here:
from statsmodels.stats.proportion import proportions_ztest
#https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html

In [16]:
# get numbers of hourly and total workers, check result 
hourly_workers_count = salaries[salaries['salary_or_hourly'] == 'Hourly'].shape[0]
print('Count hourly workers: ', hourly_workers_count)
total_workers_count = salaries.shape[0]
print('Count total workers: ', total_workers_count)
hourly_workers_count/total_workers_count

Count hourly workers:  8174
Count total workers:  33702


0.24253753486439975

> z-test with 95% confidence level (--> alpha = 0.05)  
> H0 ratio of hourly/total = 25  
> H1 ratio of hourly/total <> 25

In [17]:
# set 25% as H0
null_value = 0.25
z_stat, p_value = proportions_ztest(hourly_workers_count, total_workers_count, null_value, alternative='two-sided')

In [18]:
print("Z-statistic:", z_stat)
print("P-value:", p_value)

Z-statistic: -3.1962453870335
P-value: 0.0013922864704323116


> p-value is zero small, indicating evidence against the null hypothesis   
> further: stat is pointing in correct direction   
> **I would reject H0**